# Building a RAG chat without the langchain framework
## To understand more in detail what's going on

The technical know-how comes from Ed Donner, obviously, as well as from Sakalya Mitra & Pradip Nichite on [this gem of a blog post](https://blog.futuresmart.ai/building-rag-applications-without-langchain-or-llamaindex) I found on futuresmart.ai

In [0]:
import os
from dotenv import load_dotenv
import gradio as gr
import re
from openai import OpenAI

In [0]:
# imports for Chroma and plotly

import chromadb
from chromadb.utils import embedding_functions
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go

In [0]:
MODEL = "gpt-4o-mini"
db_name = "chroma_db"
client = chromadb.PersistentClient(path="chroma_db")

In [0]:
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
openai = OpenAI()

### Making stuff at home: documents

In [0]:
# items in documents are actually objects: Documents(metadata={...}, page_content="..."), so we need a "Document" class
# btw all the quadruple-backslash madness here is due to Windows (there might be a more efficient way, still)

class Document:
    def __init__(self, metadata, page_content):
        self.metadata = metadata
        self.page_content = page_content

    def __repr__(self):
        return f"Document(metadata={self.metadata}, page_content={repr(self.page_content)})"


documents = []

def get_documents(path='.'):
    for entry in os.listdir(path):
        if len(re.findall("^\.", entry)) == 0:
            full_path = os.path.join(path, entry)
            if os.path.isdir(full_path):
                get_documents(full_path)
            else:
                parent = re.sub("^\.[\\\\].*[\\\\]", "", os.path.dirname(full_path))
                self = os.path.basename(full_path)
                content = ""

                with open(full_path, mode="r", encoding="utf-8") as f:
                    content = f.read()
                
                doc = Document(metadata={"source": full_path, "doc_type": parent, "self": self}, page_content=content)
                documents.append(doc)

# where the knowledge collection lives
directory_path = r'.\knowledge_collection'
get_documents(directory_path)

### Making stuff at home: chunks

In [0]:
eos_pattern = re.compile(r"((?<=[.!?;])[\s]+)|([\n\r]+)")
chunk_size = 1000
chunks = []

In [0]:
for doc in documents:

    sentence_ends = [end.start() for end in list(re.finditer(eos_pattern, doc.page_content)) if end.start() > chunk_size - 50]
    start = 0
    
    if len(sentence_ends) == 0 and len(doc.page_content) > 5:
        chunk = Document(metadata=doc.metadata, page_content=doc.page_content)
        chunk.metadata['id'] = f"{doc.metadata['source']}_chunk_"
        chunks.append(chunk)

    else:        
        for point in sentence_ends:
            if point - start >= chunk_size - 50:
                text = doc.page_content[start:point]
                chunk = Document(metadata=doc.metadata, page_content=text)
                chunk.metadata['id'] = f"{doc.metadata['source']}_chunk_"
                chunks.append(chunk)
                start = point
                
        # Add the remaining part of the text as the last chunk if it's big enough
        if len(doc.page_content) - start > 5:
            text = doc.page_content[start:]
            chunk = Document(metadata=doc.metadata, page_content=text)
            chunk.metadata['id'] = f"{doc.metadata['source']}_chunk_"
            chunks.append(chunk)

### Making stuff without a framework: embeddings

In [0]:
# Configure sentence transformer embeddings
embeddings = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2"
)

collection_name = "documents_collection"

try:
    client.delete_collection(collection_name)
except ValueError:
    print(f"{collection_name} doesn't exist yet")

# Create collection
collection = client.get_or_create_collection(
    name=collection_name,
    embedding_function=embeddings
)

In [0]:
# adding our chunks to the "collection"

for chunk in chunks:
    index = chunks.index(chunk)
    collection.add(
        documents=chunk.page_content,
        metadatas=chunk.metadata,
        ids=chunk.metadata['id'] + f"{index}"
    )

In [0]:
def semantic_search(collection, query: str, n_results: int = 4):
    results = collection.query(
        query_texts=[query],
        n_results=n_results
    )
    return results

## Visualizing the Vector Store

The results actually look just as good with `all-MiniLM-L6-v2`

In [0]:
# Prework

result = collection.get(include=['embeddings', 'documents', 'metadatas'])
vectors = np.array(result['embeddings'])
documents = result['documents']
doc_types = [metadata['doc_type'] for metadata in result['metadatas']]
colors = [['blue', 'red', 'orange'][['languages', 'mountains', 'regions'].index(t)] for t in doc_types]

In [0]:
# Reduce the dimensionality of the vectors to 2D using t-SNE
# (t-distributed stochastic neighbor embedding)

tsne = TSNE(n_components=2, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 2D scatter plot
fig = go.Figure(data=[go.Scatter(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='2D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x',yaxis_title='y'),
    width=800,
    height=600,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

#### Comment: Relying on Gradio's history handling seems to be memory enough
##### If all you need is your favorite LLM with expertise in your knowlege collection

In [0]:
system_prompt = "You are a helpful assistant for everything French. Give brief, accurate answers. \
Do not provide any information that you haven't been asked for, even if you have lots of context. \
If you haven't been provided with relevant context, say you don't know. Do not make anything up, only \
provide answers that are based in the context you have been given. Do not comment on the provided context. \
If the user doesn't ask for any information, engage in brief niceties and offer your expertise regarding France."

history = [{"role": "system", "content": system_prompt}]

def get_user_prompt(prompt):
    # semantic search!!
    context = semantic_search(collection, prompt)['documents'][0]

    if len(context) > 0:
        prompt += f"\n\n[AUTOMATIC SYSTEM CONTEXT ADDITION] Here is some context that might be useful for answering the question:"

        for doc in context:
            prompt += f"\n\n{doc}"
    
    user_prompt = {"role": "user", "content": prompt}

    return user_prompt

In [0]:
def stream_gpt(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history
    messages.append(get_user_prompt(message))
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [0]:
# Gradio

gr.ChatInterface(fn=stream_gpt, type="messages").launch(inbrowser=True)